In [1]:
import re

## Functions

In [2]:
def prep_lines(lines):
    '''
    Remove Fortran-specific code, and remove \n from reading in as strings. 
    '''
    for i in range(len(lines)):
        lines[i]=lines[i].replace('     * ', '')
        lines[i]=lines[i].replace(' \n', '')
        lines[i]=lines[i].replace('\n', '')
    return lines


def read_fortran_ODEs(file, cons = False):
    '''
    Read fortran ODEs and prep for translation.\n
        - file = file name\n
        - returns: dict
    '''

    ## read fortran ODEs from given file
    with open(file, 'r') as f:
        lines = f.readlines()

    if cons == True:
        lines = lines[16:93]
    if cons == False:
        ## select non-conserved species
        lines = lines[123:]

    ## remove unnecessary fortran code
    lines = prep_lines(lines)

    ODEs = stick_fortran(lines)

    return ODEs


def stick_fortran(lines):
    '''
    Make dictionary:\n
        - entry per species\n
        - stick together separate fortran strings (result: full ODE in 1 string) \n
        \n
    This is possible by searching for 'C' in Fortran, indicating commented code.
    '''

    ODEs = dict()

    for i, line in enumerate(lines):
        if len(line) != 0 and line[0] == 'C':
            ODEs[line[2:]] = str('')            ## set species name (every that comes after the 'C')
            print('i=',i, end = '\r')
            start = i + 1                       ## start searching where this ODE ends

            for j in range(start,start+1000):
                if j >= len(lines):
                    break
                if len(lines[j]) != 0 and lines[j][0] == 'C':   ## stop searching when the next comment 'C' is found
                    stop = j
                    break
                
            for j in range(i+1,stop):           ## stick together the different pieces of fortran ODE
                ODEs[line[2:]] += lines[j]

            i = stop + 1                        ## continue after the ODE

    count = 0
    for key in ODEs:            ## count empty keys in dict()
        if key == '':
            count += 1

    while count > 0:            ## remove empty keys in dict()
        del ODEs['']
        count -= 1

    return ODEs



def fortran_to_python(ODEs, spec, nblen, nbs, cons = False, ODEtype = 'ipv'):
    '''
    Translate fortran ODE to python ODE. \n\n

    Note: NOT fractional abundances, but in units of cm^-3.
    '''
    
    print(spec)

    if isinstance(ODEs[spec],str):
        ## split string
        ODEs[spec] = ODEs[spec].split(' ')

    ## remove empty strings
    to_remove = ''
    while to_remove in ODEs[spec]:
        ODEs[spec].remove('')

    ## remove other fortran commands
    while 'END' in ODEs[spec]:
        ODEs[spec].remove('END')
    while 'RETURN' in ODEs[spec]:
        ODEs[spec].remove('RETURN')

    
    ## Translate

    ## Conserved species
    if cons == True:
        ## replace brackets: ( --> [
        print(len(ODEs[spec]))
        for i in range(len(ODEs[spec])):
            
            ODEs[spec][i] = ODEs[spec][i].replace('(', '[')
            ODEs[spec][i] = ODEs[spec][i].replace(')', ']')
            ODEs[spec][i] = ODEs[spec][i].replace('[+', '(+')
            ODEs[spec][i] = ODEs[spec][i].replace('[-', '(-')
            ODEs[spec][i] = ODEs[spec][i].replace(']]', '])')

            if ODEtype == 'torch':
                line = list(ODEs[spec][i])
                for j in range(len(line)):
                    if line[j] == 'Y':
                        line[j+1] = '[:,'

                ODEs[spec][i] = "".join(line)

    ## Non-conserved species
    if cons == False:

        ## prep for replacing final line --> sets the change in number density of species
        last = ODEs[spec][-1]
        while not last[0] == 'Y':       ## remove fortran statements & empty lines
            ODEs[spec].remove(last)
            last = ODEs[spec][-1]
        ## get reaction number
        for i in range(5,len(last)):    
            if last[i] == ')':
                stop = i
                break
        nb = last[5:stop]

        nblen[spec] = len(nb)
        nbs[spec] = int(nb)

        ## replace brackets: ( --> [
        for i in range(len(ODEs[spec])):
            if ODEs[spec][i][0] == 'F' or ODEs[spec][i][0] == 'D':
                ODEs[spec][i] = ODEs[spec][i].replace('(', '[')
                ODEs[spec][i] = ODEs[spec][i].replace(')', ']')
                ODEs[spec][i] = ODEs[spec][i].replace('F', 'F['+nb+']')
                ODEs[spec][i] = ODEs[spec][i].replace('D', 'D['+nb+']')

                if ODEtype == 'torch':
                    line = list(ODEs[spec][i])
                    for j in range(len(line)):
                        if line[j] == 'Y' or line[j] == 'F' or line[j] == 'D':
                            line[j+1] = '[:,'

                    ODEs[spec][i] = "".join(line)

        if spec == 'H':
            ODEs[spec][-1] = 'F['+nb+']=F['+nb+']+HLOSS'
        else:
            ODEs[spec][-1] = ''
             
    return ODEs


def parse_line(parsed, spec, nblen, cons = False):
    '''
    Parse line to print is too long (Python doesn't like that much).
    '''
    
    if cons == False:
        line = parsed[-1]
        
        if len(line) > 1000:
            if cons == False:
                for i in range(1000):
                    n=i+1000
                    if n < len(line):
                        if line[n] == '+' or line[n] == '-':
                            break
                    else:
                        break
                parsed[-1] = line[0:n]

                length = nblen[spec]
                
                parsed.append(line[0:length+3]+'='+line[0:length+3]+line[n:])            

        else: 
            print('No parsing need')
    
    return 

def minus_one(match):
    '''
    Helper function that is used in the regular expression substitution.
    '''
    # Extract the mached string from the matched object
    match = match.group(0)
    # remove square brackets
    match = match[1:-1]
    # Cast to integer
    match = int(match)    
    # Minus one
    match -= 1
    # Add brackets again
    match = f'[{match}]'

    
    return match

## Read in file & prep

In [3]:
# file = 'src/ode/acodes.f'
loc = 'src/ode/'
filename = 'acodes'

ODEtype = 'torch'

with open(loc+filename+'.f', 'r') as f:
    lines = f.readlines()

ODEs = read_fortran_ODEs(loc+filename+'.f')


## Translate

### Non-conserved species

In [5]:
nblen = dict()      ## dictionary to store the length of the reaction number per species
nbs = dict()
for spec in ODEs:
    ODEs = fortran_to_python(ODEs, spec, nblen, nbs, ODEtype=ODEtype)

H
H-
H+
H2+
H3+
He+
He
HeH+
C+
C
C-
CH
CH-
CH+
CH2
CH2+
N+
N
NH+
NH
CH3
CH3+
NH2+
O+
CH4+
NH2
O-
CH4
O
NH3
OH-
OH+
NH3+
CH5+
OH
NH4+
H2O
H2O+
F
H3O+
F+
HF
HF+
H2F+
Na+
Na
C2+
Mg+
Mg
C2-
C2
C2H
C2H+
C2H-
C2H2
C2H2+
CN
CN-
CN+
HNC
HCN
C2H3
C2H3+
HCN+
H2CN
CO+
Si+
N2+
Si
N2
C2H4
H2NC+
HCNH+
C2H4+
CO
SiH+
C2H5
HCO
SiH
HCO+
HOC+
C2H5+
N2H+
CH2NH
H2CO+
SiH2
CH3CH3
NO
SiH2+
CH4N+
NO+
CH2NH2+
H2CO
CH3CH3+
SiH3
SiH3+
H3CO+
C2H7+
P
P+
HNO
HNO+
CF+
S+
SiH4
O2-
O2+
SiH4+
S
CH3OH+
S-
PH
H2NO+
PH+
CH3OH
O2
O2H
PH2+
O2H+
SiH5+
CH3OH2+
HS+
PH2
HS
H2S+
PH3+
H2O2
H2S
H3S+
Cl
Cl+
C3+
C3
C3-
HCl
HCl+
C3H
H2Cl+
C3H-
C3H+
C3H2
C3H2+
C2N+
H2CCC
CNC+
C2N
CH2CCH+
C2NH+
CH2CCH
C3H3+
CH2CCH2
C2O+
CH2CN
CH2CN+
C2O
C3H4+
CH3CCH
SiC+
SiC
C3H5+
CH3CN
CH3CN+
HC2O+
HCSi
HCSi+
CH2CO+
OCN+
SiCH2+
NH2CN
C3H6+
CH3CHCH2
CNO
CH2CO
SiN+
SiCH2
OCN
CH3CNH+
SiN
C3H7+
SiCH3
CP+
HNSi+
HNSi
HCNO+
HNCO
SiCH3+
CH3CO+
HCNO
HONC+
CP
HNCO+
HOCN+
NH2CNH+
HONC
HOCN
N2O+
SiO+
HCNOH+
CS+
H2CNO+
H2NCO+
CO2+
SiCH4+
N2O
HCP+
H2OCN+
CH3CHO+
HN

In [6]:
## Parsing the ODEs of the non-conserved specs
for spec in ODEs:
    print(spec)

    i = 0
    while i < len(ODEs[spec]):

        if len(ODEs[spec][i]) > 1000:
            parsed = [ODEs[spec][i]]
            while len(parsed[-1]) > 1000:
                parse_line(parsed, spec, nblen)

            ODEs[spec] = ODEs[spec][0:i] + parsed + ODEs[spec][i+1:]

            i = i + len(parsed)
        

        else: i += 1

H
H-
H+
H2+
H3+
He+
He
HeH+
C+
C
C-
CH
CH-
CH+
CH2
CH2+
N+
N
NH+
NH
CH3
CH3+
NH2+
O+
CH4+
NH2
O-
CH4
O
NH3
OH-
OH+
NH3+
CH5+
OH
NH4+
H2O
H2O+
F
H3O+
F+
HF
HF+
H2F+
Na+
Na
C2+
Mg+
Mg
C2-
C2
C2H
C2H+
C2H-
C2H2
C2H2+
CN
CN-
CN+
HNC
HCN
C2H3
C2H3+
HCN+
H2CN
CO+
Si+
N2+
Si
N2
C2H4
H2NC+
HCNH+
C2H4+
CO
SiH+
C2H5
HCO
SiH
HCO+
HOC+
C2H5+
N2H+
CH2NH
H2CO+
SiH2
CH3CH3
NO
SiH2+
CH4N+
NO+
CH2NH2+
H2CO
CH3CH3+
SiH3
SiH3+
H3CO+
C2H7+
P
P+
HNO
HNO+
CF+
S+
SiH4
O2-
O2+
SiH4+
S
CH3OH+
S-
PH
H2NO+
PH+
CH3OH
O2
O2H
PH2+
O2H+
SiH5+
CH3OH2+
HS+
PH2
HS
H2S+
PH3+
H2O2
H2S
H3S+
Cl
Cl+
C3+
C3
C3-
HCl
HCl+
C3H
H2Cl+
C3H-
C3H+
C3H2
C3H2+
C2N+
H2CCC
CNC+
C2N
CH2CCH+
C2NH+
CH2CCH
C3H3+
CH2CCH2
C2O+
CH2CN
CH2CN+
C2O
C3H4+
CH3CCH
SiC+
SiC
C3H5+
CH3CN
CH3CN+
HC2O+
HCSi
HCSi+
CH2CO+
OCN+
SiCH2+
NH2CN
C3H6+
CH3CHCH2
CNO
CH2CO
SiN+
SiCH2
OCN
CH3CNH+
SiN
C3H7+
SiCH3
CP+
HNSi+
HNSi
HCNO+
HNCO
SiCH3+
CH3CO+
HCNO
HONC+
CP
HNCO+
HOCN+
NH2CNH+
HONC
HOCN
N2O+
SiO+
HCNOH+
CS+
H2CNO+
H2NCO+
CO2+
SiCH4+
N2O
HCP+
H2OCN+
CH3CHO+
HN

### Conserved species

In [7]:
ODEs_cons = read_fortran_ODEs(loc+filename+'.f', True)

In [8]:
for spec in ODEs_cons:
    ODEs_cons = fortran_to_python(ODEs_cons, spec, nblen, nbs, cons = True, ODEtype=ODEtype)

e-
1
H2
1


## Write out in as python function

In [12]:

loc = 'src/ode/'

if ODEtype == 'torch':
    filename = filename+'_torch'

print(loc+filename+'.py')

## Y = number density of unconserved species
## X = number density of conserved species
## K = reaction rate
## ACCR = loss of H due to dust interaction, kind of rate

with open(loc+filename+'.py', 'w') as f:
    f.write('import numpy as np\n')
    if filename == 'acodes' or filename =='acodes_torch' :
        f.write("\n'''\nIMPORTANT NOTE:\nThe content of this function is translated from Fortran 77. \n \nWritten by Silke Maes, April 2023.\nOriginal Fortran code by rate10odecsT.pl on Sun, Jul 21 2019, \nusing ratefile: rate16_IP_2330K_AP_6000K.rates, from http://udfa.ajmarkwick.net/index.php. \n'''\n")
    if filename == 'dcodes' or filename =='dcodes_torch':
        f.write("\n'''\nIMPORTANT NOTE:\nThe content of this function is translated from Fortran 77. \n \nWritten by Silke Maes, April 2023.\nOriginal Fortran code by rate13dc.pl on Fri, May 31 2013, \nusing ratefile: rate13.rates, from http://udfa.ajmarkwick.net/index.php. \n'''\n")
    f.write("\n")

    ## start of the function
    if ODEtype == 'ivp':
        f.write('\ndef ODE(t, Y, YDOT, X, TOTAL, K, HNR, ACCR):\n')
    if ODEtype == 'torch':
        f.write('\ndef torchODE(t, Y, args):\n')
    
    ## documentation
    f.write("\t'''")
    f.write("\n\tCalculate the ODEs.\n")
    f.write("\n\tY     = number density of the non-conserved species                      == n")
    f.write("\n\tYDOT  = change in number density of the conserved species                == ndot")
    f.write("\n\tX     = number density of the conserved species during calculation       == nconsvdot")
    f.write("\n\tK     = reaction rates, calculated in rates.py                           == k ")
    f.write("\n\tACCR  = H accretion onto dust, calculated in main.py from input physics  == Haccr ")
    f.write("\n\tTOTAL = initial number density of conserved species                      == nconsv")
    f.write("\n\tHNR   = input density                                                    == ρ")
    f.write("\n\t'''\n")

    ## set cut off
    f.write("\n\tcutoff = 1e-50\n")
    f.write("\n\tY[Y < cutoff] = 0\n")

    if ODEtype == 'torch':
        f.write("\n\tYDOT, X, TOTAL, K, HNR, ACCR = args")
        f.write("\n\n\tYDOT = torch.from_numpy(YDOT)")
        f.write("\n\tX = torch.from_numpy(X)")
        f.write("\n\tTOTAL = torch.from_numpy(TOTAL)")
        f.write("\n\tK = torch.from_numpy(K)")

    ## conserved species
    f.write('\n\t## Conserved species\n')
    for key in ODEs_cons:
        f.write('\n\t## '+str(key)+'\n')
        for line in ODEs_cons[key]:
            f.write('\t'+line+'\n')

    ## non-conserved species
    f.write('\n\n\t## Non-conserved species\n')
    if ODEtype == 'ivp':
        f.write('\tHLOSS=-ACCR*Y[1]\n')
        f.write('\tF = np.zeros(len(Y))\n')
        f.write('\tD = np.zeros(len(Y))\n')
    if ODEtype == 'torch':
        f.write('\tHLOSS=-ACCR*Y[:,1]\n')
        f.write('\tF = torch.zeros(Y.shape[1]).view(1,-1)\n')
        f.write('\tD = torch.zeros(Y.shape[1]).view(1,-1)\n')
    for key in ODEs:
        f.write('\n\t## '+str(key)+'\n')
        for line in ODEs[key]:
            f.write('\t'+line+'\n')

    ## return
    f.write('\n\tYDOT=F-(D*Y)\n')
    f.write('\n\treturn YDOT')

# Read the original ODE file
with open(loc+filename+'.py', 'r') as file:
    content = file.read()

# Substitute every number between square brackets with that number minus one
content_new = re.sub('\[\d+\]', minus_one, content)

# Write the new ODE file
with open(loc+filename+'.py', 'w') as file:
    file.write(content_new)

src/ode/dcodes.py


'''
acodes.py: ODEs for the conserved species, manually parsed --> plug in final file
'''


## e-
    T = 0.+Y[1]-Y[2]+Y[3]+Y[4]+Y[6]+Y[7]-Y[8]+Y[9]-Y[11]+Y[12]+Y[14]+Y[15]+Y[20]+Y[21]+Y[22]+Y[23]+Y[24]-Y[25]-Y[29]+Y[31]+Y[32]+Y[33]+Y[35]+Y[37]+Y[39]+Y[40]+Y[42]+Y[43]+Y[45]-Y[46]+Y[47]+Y[49]-Y[51]+Y[52]+Y[56]-Y[57]+Y[58]+Y[61]+Y[62]+Y[65]+Y[66]+Y[68]+Y[69]+Y[72]+Y[74]+Y[75]+Y[77]+Y[78]+Y[82]+Y[83]+Y[84]+Y[86]+Y[89]+Y[90]+Y[92]+Y[93]+Y[94]+Y[96]+Y[97]+Y[98]+Y[100]+Y[101]+Y[107]+Y[109]+Y[110]-Y[111]-Y[112]+Y[113]+Y[114]+Y[115]+Y[116]+Y[118]+Y[120]+Y[122]+Y[123]+Y[125]+Y[127]+Y[128]+Y[130]+Y[131]-Y[133]+Y[135]+Y[136]-Y[137]+Y[138]+Y[141]+Y[142]+Y[144]+Y[146]+Y[147]+Y[149]+Y[152]+Y[154]+Y[155]+Y[158]+Y[159]+Y[161]+Y[162]+Y[163]+Y[165]+Y[167]+Y[170]+Y[171]+Y[174]+Y[176]+Y[178]+Y[180]+Y[181]+Y[183]+Y[186]+Y[187]+Y[188]+Y[191]+Y[192]+Y[194]+Y[195]+Y[196]+Y[197]+Y[199]+Y[200]+Y[201]+Y[202]+Y[203]+Y[205]+Y[208]+Y[209]+Y[211]+Y[212]+Y[214]+Y[215]+Y[216]+Y[217]+Y[220]+Y[221]+Y[222]+Y[223]+Y[228]+Y[229]+Y[232]+Y[234]+Y[235]+Y[236]+Y[238]+Y[239]+Y[240]+Y[241]+Y[242]+Y[244]+Y[245]+Y[246]+Y[247]+Y[248]+Y[249]+Y[251]+Y[252]-Y[253]
    T = T+Y[254]+Y[258]+Y[259]+Y[260]+Y[261]+Y[262]+Y[263]-Y[265]+Y[266]+Y[267]+Y[270]-Y[271]+Y[272]+Y[276]+Y[277]+Y[282]+Y[284]+Y[285]+Y[286]+Y[287]+Y[288]+Y[289]+Y[292]+Y[293]+Y[294]+Y[295]+Y[296]+Y[298]+Y[300]+Y[302]+Y[305]+Y[306]+Y[307]+Y[308]+Y[310]+Y[311]+Y[312]+Y[314]+Y[316]+Y[317]+Y[318]+Y[320]+Y[321]+Y[322]+Y[323]+Y[326]-Y[328]+Y[329]+Y[332]+Y[333]+Y[334]+Y[335]+Y[336]+Y[337]+Y[339]-Y[340]+Y[341]+Y[342]+Y[345]+Y[346]+Y[350]+Y[351]+Y[352]+Y[354]+Y[355]+Y[357]+Y[358]+Y[359]+Y[360]+Y[363]+Y[364]+Y[365]+Y[366]+Y[368]+Y[369]+Y[371]+Y[373]+Y[374]-Y[375]+Y[376]+Y[379]-Y[380]+Y[382]-Y[384]+Y[385]+Y[386]+Y[388]+Y[389]+Y[390]+Y[391]+Y[393]+Y[394]+Y[395]+Y[397]+Y[399]+Y[400]+Y[402]+Y[403]+Y[404]+Y[405]-Y[407]-Y[408]+Y[409]+Y[411]+Y[413]+Y[414]+Y[417]+Y[418]-Y[419]+Y[421]+Y[423]-Y[424]+Y[427]+Y[428]+Y[430]+Y[431]+Y[432]+Y[433]+Y[434]+Y[435]+Y[437]-Y[438]+Y[439]-Y[440]+Y[442]+Y[444]+Y[445]+Y[447]+Y[448]-Y[449]+Y[451]+Y[452]-Y[453]+Y[456]+Y[458]+Y[459]+Y[460]+Y[462]+Y[463]+Y[465]
    X[0]=0.+TOTAL[0]+T

    ## H2
    T = 0.+Y[0]+Y[1]+Y[2]+2*Y[3]+3*Y[4]+Y[7]+Y[11]+Y[12]+Y[13]+2*Y[14]+2*Y[16]+3*Y[18]+Y[19]+Y[20]+3*Y[21]+2*Y[23]+4*Y[24]+2*Y[26]+4*Y[28]+Y[29]+Y[30]+Y[31]+5*Y[32]+3*Y[33]+3*Y[34]+4*Y[35]+2*Y[36]+2*Y[37]+3*Y[39]+Y[41]+Y[42]+2*Y[43]+Y[51]+Y[52]+Y[53]+2*Y[55]+2*Y[56]+Y[59]+3*Y[60]+3*Y[61]+Y[62]+Y[63]+4*Y[65]+2*Y[67]+2*Y[68]+4*Y[73]+2*Y[74]+5*Y[75]+Y[76]+Y[77]+Y[78]+3*Y[79]+Y[80]+5*Y[81]+Y[82]+Y[83]+4*Y[84]+2*Y[85]+6*Y[86]+2*Y[88]+4*Y[89]+2*Y[90]+6*Y[91]+2*Y[92]+3*Y[96]+Y[97]+3*Y[98]+Y[99]+7*Y[101]+3*Y[102]+4*Y[104]+Y[105]+2*Y[107]+4*Y[108]+4*Y[109]+Y[113]+4*Y[114]+5*Y[116]+Y[117]+2*Y[118]+Y[119]+Y[120]+2*Y[121]+Y[122]+5*Y[123]+2*Y[124]+2*Y[125]+2*Y[126]+3*Y[127]+3*Y[128]+Y[132]+Y[135]+2*Y[136]+Y[137]+Y[138]+Y[139]+2*Y[141]+2*Y[143]+2*Y[145]+Y[146]+3*Y[147]+3*Y[148]+3*Y[149]+4*Y[151]+4*Y[154]+2*Y[156]+4*Y[157]+2*Y[158]+5*Y[159]+Y[160]+Y[161]+3*Y[162]+Y[163]+3*Y[164]+6*Y[165]+4*Y[167]+2*Y[168]+6*Y[169]+2*Y[171]+2*Y[172]+2*Y[174]+2*Y[177]+Y[179]+Y[180]+Y[181]+Y[182]+3*Y[183]+Y[184]+7*Y[186]+3*Y[187]+Y[188]+Y[189]+Y[190]
    T = T+3*Y[191]+Y[192]+3*Y[193]+Y[194]+2*Y[196]+4*Y[199]+2*Y[201]+2*Y[202]+2*Y[203]+Y[204]+4*Y[206]+2*Y[208]+4*Y[209]+2*Y[211]+Y[212]+5*Y[214]+Y[215]+Y[216]+Y[219]+2*Y[220]+Y[221]+Y[222]+2*Y[224]+2*Y[225]+3*Y[227]+2*Y[228]+6*Y[229]+6*Y[231]+2*Y[232]+2*Y[233]+3*Y[234]+2*Y[235]+6*Y[236]+6*Y[237]+Y[239]+7*Y[240]+2*Y[242]+3*Y[244]+Y[247]+4*Y[248]+3*Y[249]+7*Y[251]+3*Y[252]+Y[254]+Y[255]+3*Y[260]+Y[261]+2*Y[262]+2*Y[263]+Y[264]+Y[265]+Y[266]+2*Y[267]+2*Y[269]+Y[272]+3*Y[273]+Y[274]+3*Y[277]+Y[278]+4*Y[280]+2*Y[284]+4*Y[285]+Y[288]+5*Y[289]+Y[290]+3*Y[291]+3*Y[292]+Y[293]+Y[294]+2*Y[295]+4*Y[296]+6*Y[297]+2*Y[298]+2*Y[299]+Y[302]+5*Y[304]+3*Y[305]+7*Y[307]+3*Y[308]+Y[309]+Y[310]+6*Y[312]+Y[316]+2*Y[317]+6*Y[318]+6*Y[319]+3*Y[320]+7*Y[321]+3*Y[322]+4*Y[323]+4*Y[324]+4*Y[333]+Y[334]+Y[335]+Y[336]+Y[337]+Y[338]+5*Y[339]+Y[340]+2*Y[341]+2*Y[343]+Y[345]+3*Y[346]+4*Y[347]+4*Y[351]+2*Y[354]+Y[356]+Y[357]+Y[358]+3*Y[359]+Y[360]+3*Y[361]+Y[362]+5*Y[363]+4*Y[364]+2*Y[365]+2*Y[366]+2*Y[367]+3*Y[368]+3*Y[369]+Y[371]+Y[374]+Y[378]
    T = T+Y[379]+Y[380]+2*Y[381]+2*Y[382]+Y[386]+Y[387]+3*Y[388]+4*Y[390]+2*Y[391]+3*Y[393]+Y[394]+5*Y[395]+6*Y[396]+6*Y[397]+7*Y[399]+Y[402]+Y[404]+Y[408]+Y[409]+Y[410]+2*Y[411]+2*Y[412]+3*Y[413]+4*Y[414]+4*Y[415]+3*Y[416]+5*Y[417]+4*Y[418]+Y[422]+Y[423]+Y[424]+2*Y[425]+2*Y[428]+Y[429]+Y[430]+3*Y[431]+4*Y[432]+2*Y[433]+5*Y[434]+3*Y[435]+Y[439]+Y[440]+Y[441]+2*Y[442]+2*Y[443]+3*Y[444]+4*Y[445]+3*Y[446]+5*Y[447]+4*Y[448]+Y[452]+Y[453]+Y[454]+2*Y[457]+2*Y[458]+Y[459]+3*Y[460]+Y[461]+2*Y[462]+3*Y[463]
    X[1]=0.+TOTAL[1]-0.5*T

''' 
dcodes.py: ODEs for the conserved species, manually parsed --> plug in final file (dc fortran code) 
'''

## e-
	T=0.-Y[1]+Y[2]+Y[3]+Y[4]+Y[5]+Y[7]+Y[8]-Y[10]-Y[12]+Y[13]+Y[15]+Y[16]+Y[18]+Y[21]+Y[22]+Y[23]+Y[24]-Y[26]-Y[30]+Y[31]+Y[32]+Y[33]+Y[35]+Y[37]+Y[39]+Y[40]+Y[42]+Y[43]+Y[44]+Y[46]+Y[47]-Y[49]+Y[52]-Y[53]+Y[55]-Y[57]+Y[58]+Y[62]+Y[63]+Y[65]+Y[66]+Y[67]+Y[71]+Y[72]+Y[73]+Y[75]+Y[79]+Y[80]+Y[81]+Y[82]+Y[84]+Y[88]+Y[89]+Y[90]+Y[91]+Y[93]+Y[95]+Y[96]+Y[97]+Y[99]+Y[101]+Y[102]+Y[103]-Y[105]+Y[106]+Y[107]+Y[109]-Y[110]+Y[112]+Y[113]+Y[117]+Y[118]+Y[119]+Y[120]+Y[121]+Y[124]+Y[125]+Y[128]+Y[130]+Y[131]-Y[133]+Y[135]+Y[137]-Y[138]+Y[139]+Y[141]+Y[142]+Y[144]+Y[146]+Y[147]+Y[149]+Y[151]+Y[153]+Y[155]+Y[157]+Y[159]+Y[161]+Y[162]+Y[164]+Y[165]+Y[166]+Y[167]+Y[169]+Y[173]+Y[176]+Y[178]+Y[180]+Y[181]+Y[183]+Y[185]+Y[186]+Y[188]+Y[190]+Y[191]+Y[192]+Y[195]+Y[196]+Y[197]+Y[198]+Y[199]+Y[200]+Y[201]+Y[202]+Y[204]+Y[205]+Y[206]+Y[207]+Y[208]+Y[215]+Y[216]+Y[217]+Y[219]+Y[220]+Y[221]+Y[222]+Y[223]+Y[224]+Y[225]+Y[227]+Y[229]+Y[230]+Y[232]+Y[237]+Y[239]+Y[240]+Y[241]+Y[244]+Y[245]+Y[246]+Y[247]+Y[248]+Y[249]
	T=T+Y[250]+Y[251]+Y[252]+Y[253]+Y[254]+Y[257]+Y[258]-Y[259]+Y[261]+Y[262]+Y[264]-Y[265]+Y[266]+Y[267]-Y[268]+Y[271]+Y[274]+Y[275]+Y[278]+Y[279]+Y[281]+Y[284]+Y[285]+Y[286]+Y[288]+Y[289]+Y[291]+Y[292]+Y[293]+Y[295]+Y[297]+Y[298]+Y[299]+Y[302]+Y[304]+Y[305]+Y[306]+Y[307]+Y[310]+Y[312]+Y[313]+Y[315]+Y[316]+Y[317]+Y[318]+Y[319]+Y[321]+Y[322]+Y[323]+Y[324]+Y[326]+Y[327]+Y[329]-Y[333]+Y[334]-Y[336]+Y[337]+Y[338]+Y[339]+Y[340]+Y[342]+Y[343]+Y[345]+Y[346]+Y[348]+Y[350]+Y[351]+Y[352]+Y[355]+Y[357]+Y[358]+Y[360]+Y[361]+Y[363]+Y[364]+Y[365]+Y[366]+Y[369]+Y[370]+Y[371]+Y[372]+Y[374]-Y[375]+Y[376]+Y[379]-Y[380]+Y[381]-Y[383]+Y[385]+Y[387]+Y[388]+Y[389]+Y[390]+Y[391]+Y[393]+Y[394]+Y[395]+Y[396]+Y[398]+Y[400]+Y[401]+Y[402]+Y[404]+Y[405]-Y[407]+Y[408]-Y[409]+Y[412]+Y[413]+Y[415]+Y[417]+Y[418]+Y[420]-Y[421]+Y[422]-Y[423]+Y[425]+Y[428]+Y[429]+Y[430]+Y[432]+Y[433]+Y[434]+Y[435]+Y[436]-Y[437]-Y[439]+Y[440]+Y[443]+Y[444]+Y[445]+Y[446]+Y[448]+Y[449]-Y[450]-Y[452]+Y[454]+Y[457]+Y[458]+Y[459]+Y[460]+Y[462]+Y[463]+Y[465]
	X[0]=0.+TOTAL[0]+T

	## H2
	T=0.+Y[0]+Y[1]+Y[2]+2*Y[3]+3*Y[4]+Y[7]+Y[11]+Y[12]+Y[13]+2*Y[14]+2*Y[15]+Y[18]+Y[19]+3*Y[20]+3*Y[21]+2*Y[22]+4*Y[24]+2*Y[25]+4*Y[27]+3*Y[29]+Y[30]+Y[31]+3*Y[32]+5*Y[33]+Y[34]+4*Y[35]+2*Y[36]+2*Y[37]+3*Y[39]+Y[41]+Y[42]+2*Y[43]+Y[51]+Y[52]+Y[53]+2*Y[54]+2*Y[55]+Y[59]+Y[60]+3*Y[61]+3*Y[62]+Y[63]+2*Y[64]+4*Y[70]+2*Y[71]+2*Y[72]+4*Y[73]+Y[75]+5*Y[76]+Y[77]+Y[78]+Y[79]+Y[80]+5*Y[81]+Y[82]+3*Y[83]+2*Y[84]+2*Y[85]+6*Y[86]+2*Y[88]+4*Y[89]+4*Y[91]+2*Y[92]+6*Y[93]+3*Y[94]+3*Y[95]+3*Y[96]+7*Y[97]+Y[100]+Y[101]+4*Y[104]+4*Y[107]+4*Y[109]+Y[111]+2*Y[112]+Y[113]+4*Y[114]+Y[116]+2*Y[117]+Y[118]+5*Y[119]+5*Y[120]+Y[121]+2*Y[122]+Y[123]+2*Y[124]+3*Y[125]+2*Y[126]+2*Y[127]+3*Y[128]+Y[134]+Y[135]+Y[136]+2*Y[137]+Y[138]+Y[139]+2*Y[140]+2*Y[141]+2*Y[143]+3*Y[146]+Y[147]+3*Y[148]+3*Y[149]+4*Y[150]+2*Y[152]+2*Y[153]+4*Y[155]+4*Y[156]+5*Y[159]+3*Y[160]+3*Y[161]+Y[162]+Y[163]+Y[164]+2*Y[165]+2*Y[167]+2*Y[168]+6*Y[169]+6*Y[170]+2*Y[172]+2*Y[174]+4*Y[176]+7*Y[178]+3*Y[179]+Y[181]+Y[182]+Y[183]+Y[184]+3*Y[185]+3*Y[186]
	T=T+Y[187]+Y[188]+Y[190]+Y[191]+3*Y[192]+Y[193]+Y[194]+2*Y[197]+2*Y[199]+2*Y[200]+4*Y[202]+Y[204]+2*Y[205]+4*Y[206]+2*Y[207]+2*Y[208]+Y[210]+4*Y[211]+Y[214]+Y[215]+Y[216]+Y[217]+5*Y[220]+Y[221]+2*Y[222]+Y[223]+2*Y[224]+6*Y[225]+6*Y[226]+3*Y[227]+6*Y[228]+2*Y[230]+3*Y[231]+2*Y[233]+2*Y[234]+2*Y[237]+2*Y[238]+6*Y[239]+Y[240]+3*Y[241]+2*Y[245]+4*Y[246]+3*Y[249]+7*Y[250]+7*Y[251]+3*Y[252]+Y[253]+Y[256]+3*Y[257]+Y[261]+2*Y[262]+Y[263]+2*Y[264]+Y[265]+Y[266]+2*Y[270]+2*Y[271]+3*Y[272]+Y[274]+Y[276]+Y[277]+3*Y[278]+4*Y[280]+2*Y[281]+4*Y[284]+5*Y[288]+Y[289]+3*Y[290]+Y[291]+3*Y[292]+Y[293]+Y[294]+6*Y[296]+2*Y[297]+4*Y[298]+2*Y[299]+2*Y[301]+5*Y[303]+3*Y[304]+3*Y[305]+7*Y[306]+Y[307]+6*Y[312]+Y[314]+Y[315]+Y[316]+2*Y[317]+6*Y[318]+3*Y[319]+6*Y[320]+3*Y[321]+4*Y[322]+7*Y[323]+4*Y[324]+4*Y[328]+Y[334]+Y[335]+Y[336]+Y[337]+Y[338]+Y[339]+5*Y[340]+2*Y[342]+2*Y[344]+3*Y[345]+Y[346]+2*Y[348]+4*Y[351]+4*Y[353]+Y[356]+5*Y[357]+Y[358]+3*Y[359]+Y[360]+3*Y[361]+Y[362]+Y[363]+2*Y[364]+4*Y[365]+2*Y[366]+2*Y[367]
	T=T+3*Y[369]+3*Y[370]+Y[371]+Y[374]+Y[378]+Y[379]+Y[380]+2*Y[382]+2*Y[385]+Y[386]+Y[387]+3*Y[388]+2*Y[389]+4*Y[390]+Y[393]+5*Y[394]+3*Y[395]+6*Y[396]+6*Y[397]+7*Y[398]+Y[401]+Y[404]+Y[408]+Y[409]+Y[410]+2*Y[411]+2*Y[412]+3*Y[413]+4*Y[414]+4*Y[415]+3*Y[416]+5*Y[417]+4*Y[418]+Y[422]+Y[423]+Y[424]+2*Y[427]+2*Y[428]+Y[429]+3*Y[430]+Y[431]+4*Y[432]+2*Y[433]+3*Y[434]+5*Y[435]+Y[439]+Y[440]+Y[441]+2*Y[442]+2*Y[443]+3*Y[444]+4*Y[445]+5*Y[446]+3*Y[447]+4*Y[448]+Y[452]+Y[453]+Y[454]+2*Y[456]+2*Y[457]+Y[459]+3*Y[460]+Y[461]+2*Y[462]+3*Y[463]
	X[1]=0.+TOTAL[1]-0.5*T

# Translate to torch


In [14]:
## testing

str1 = 'F[3]=0.+K[456]*X[1]*Y[40]*HNR+K[457]*X[1]*Y[6]*HNR+K[730]*X[1]+K[2564]*Y[1]*Y[76]*HNR+K[3064]*Y[0]*Y[7]*HNR+K[5921]*Y[4]+K[6092]*Y[1]*Y[0]*HNR+K[6396]*Y[4]+K[6732]*Y[4]'
str2 = 'D[3]=0.+K[435]*Y[50]*HNR+K[436]*Y[55]*HNR+K[437]*Y[73]*HNR+K[438]*Y[53]*HNR+K[439]*Y[16]*HNR+K[440]*Y[91]*HNR+K[441]*Y[28]*HNR+K[442]*Y[13]*HNR+K[443]*Y[54]*HNR+K[444]*Y[71]*HNR+K[445]*Y[88]*HNR+K[446]*Y[36]*HNR+K[447]*Y[126]*HNR+K[448]*Y[59]*HNR+K[449]*Y[76]*HNR+K[450]*Y[26]*HNR+K[451]*Y[34]*HNR+K[452]*Y[19]*HNR+K[453]*Y[87]*HNR+K[454]*Y[103]*HNR+K[455]*Y[30]*HNR+K[488]*Y[0]*HNR+K[1238]*X[0]*HNR+K[2588]*Y[50]*HNR+K[2589]*Y[55]*HNR+K[2590]*Y[73]*HNR+K[2591]*Y[73]*HNR+K[2592]*Y[53]*HNR+K[2593]*Y[10]*HNR+K[2594]*Y[16]*HNR+K[2595]*Y[91]*HNR+K[2596]*Y[91]*HNR+K[2597]*Y[28]*HNR+K[2598]*Y[28]*HNR+K[2599]*Y[13]*HNR+K[2600]*Y[54]*HNR+K[2601]*Y[213]*HNR+K[2602]*Y[71]*HNR+K[2603]*X[1]*HNR+K[2604]*Y[88]*HNR+K[2605]*Y[36]*HNR+K[2606]*Y[126]*HNR+K[2607]*Y[126]*HNR+K[2608]*Y[76]*HNR+K[2609]*Y[5]*HNR+K[2610]*Y[64]*HNR+K[2611]*Y[17]*HNR+K[2612]*Y[19]*HNR+K[2613]*Y[87]*HNR+K[2614]*Y[103]*HNR+K[2615]*Y[27]*HNR+K[2616]*Y[30]*HNR+K[4921]*Y[2]*HNR+K[5902]+K[6377]+K[6713]'
	
str1 = list(str1)

for i in range(len(str1)):
    if str1[i] == 'Y':
        str1[i+1] = '[:,'
    
str1 = "".join(str1)

In [15]:
str1

'F[3]=0.+K[456]*X[1]*Y[:,40]*HNR+K[457]*X[1]*Y[:,6]*HNR+K[730]*X[1]+K[2564]*Y[:,1]*Y[:,76]*HNR+K[3064]*Y[:,0]*Y[:,7]*HNR+K[5921]*Y[:,4]+K[6092]*Y[:,1]*Y[:,0]*HNR+K[6396]*Y[:,4]+K[6732]*Y[:,4]'